# Enter Information

## Check / Create Tables

In [3]:
def check_create_tables(cursor):
    # Create DEPARTMENT table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS DEPARTMENT (
            dept_code VARCHAR(4),
            name VARCHAR(100) NOT NULL,
            PRIMARY KEY (dept_code)
        );
    """)

    # Create DEGREE table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS DEGREE (
            dept_code VARCHAR(4),
            name VARCHAR(100),
            level VARCHAR(10),
            PRIMARY KEY (dept_code, name, level),
            FOREIGN KEY (dept_code) REFERENCES DEPARTMENT(dept_code)
        );
    """)

    # Create COURSE table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS COURSE (
            course_number VARCHAR(8),
            dept_code VARCHAR(4),
            name VARCHAR(100) NOT NULL,
            PRIMARY KEY (course_number),
            FOREIGN KEY (dept_code) REFERENCES DEPARTMENT(dept_code)
        );
    """)

    # Create DEGREE_COURSE table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS DEGREE_COURSE (
            dept_code VARCHAR(4),
            degree_name VARCHAR(100),
            degree_level VARCHAR(10),
            course_number VARCHAR(8),
            is_core BOOLEAN DEFAULT FALSE,
            PRIMARY KEY (dept_code, degree_name, degree_level, course_number),
            FOREIGN KEY (dept_code, degree_name, degree_level) 
                REFERENCES DEGREE(dept_code, name, level),
            FOREIGN KEY (course_number) 
                REFERENCES COURSE(course_number)
        );
    """)

    # Create GOAL table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS GOAL (
            dept_code VARCHAR(4),
            degree_name VARCHAR(100),
            degree_level VARCHAR(10),
            code CHAR(4),
            description TEXT NOT NULL,
            PRIMARY KEY (dept_code, degree_name, degree_level, code),
            FOREIGN KEY (dept_code, degree_name, degree_level) 
                REFERENCES DEGREE(dept_code, name, level)
        );
    """)

    # Create SEMESTER table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS SEMESTER (
            year INT,
            term ENUM('Spring', 'Summer', 'Fall'),
            PRIMARY KEY (year, term)
        );
    """)

    # Create INSTRUCTOR table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS INSTRUCTOR (
            instructor_id CHAR(8),
            name VARCHAR(100) NOT NULL,
            dept_code VARCHAR(4),
            PRIMARY KEY (instructor_id),
            FOREIGN KEY (dept_code) REFERENCES DEPARTMENT(dept_code)
        );
    """)

    # Create SECTION table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS SECTION (
            course_number VARCHAR(8),
            year INT,
            term ENUM('Spring', 'Summer', 'Fall'),
            section_number CHAR(3),
            enrollment INT NOT NULL DEFAULT 0,
            instructor_id CHAR(8) NOT NULL,
            PRIMARY KEY (course_number, year, term, section_number),
            FOREIGN KEY (course_number) REFERENCES COURSE(course_number),
            FOREIGN KEY (year, term) REFERENCES SEMESTER(year, term),
            FOREIGN KEY (instructor_id) REFERENCES INSTRUCTOR(instructor_id)
        );
    """)

    # Create EVALUATION table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS EVALUATION (
            course_number VARCHAR(8),
            year INT,
            term ENUM('Spring', 'Summer', 'Fall'),
            section_number CHAR(3),
            dept_code VARCHAR(4),
            degree_name VARCHAR(100),
            degree_level VARCHAR(10),
            goal_code CHAR(4),
            evaluation_method VARCHAR(50) NOT NULL,
            num_a INT NOT NULL DEFAULT 0,
            num_b INT NOT NULL DEFAULT 0,
            num_c INT NOT NULL DEFAULT 0,
            num_f INT NOT NULL DEFAULT 0,
            improvement_notes TEXT,
            PRIMARY KEY (course_number, year, term, section_number, dept_code, degree_name, degree_level, goal_code),
            FOREIGN KEY (course_number, year, term, section_number) 
                REFERENCES SECTION(course_number, year, term, section_number),
            FOREIGN KEY (dept_code, degree_name, degree_level, goal_code) 
                REFERENCES GOAL(dept_code, degree_name, degree_level, code),
            FOREIGN KEY (dept_code, degree_name, degree_level, course_number)
                REFERENCES DEGREE_COURSE(dept_code, degree_name, degree_level, course_number)
        );
    """)


## Clear All Data

In [5]:
def clear_data(cursor):
    
    cursor.execute("DELETE FROM EVALUATION;")
    cursor.execute("DELETE FROM SECTION;")
    cursor.execute("DELETE FROM INSTRUCTOR;")
    cursor.execute("DELETE FROM DEGREE_COURSE;")
    cursor.execute("DELETE FROM COURSE;")
    cursor.execute("DELETE FROM GOAL;")
    cursor.execute("DELETE FROM DEGREE;")
    cursor.execute("DELETE FROM SEMESTER;")
    cursor.execute("DELETE FROM DEPARTMENT;")


## Enter Degree

In [7]:
import re
import sys

def add_degree(cursor):
    """
    Allows the user to add a new degree program and creates the department if not found.
    """
    try:
        # Start a transaction
        cursor.execute("BEGIN;")
        
        # Prompt user for department code
        dept_code = input("Enter the department code (2-4 characters): ").strip().upper()
        if not re.fullmatch(r'[A-Z]{2,4}', dept_code):
            print("Error: Department code must be 2-4 uppercase characters.")
            cursor.execute("ROLLBACK;")
            return
        
        # Check if department exists
        cursor.execute("SELECT COUNT(*) FROM DEPARTMENT WHERE dept_code = %s;", (dept_code,))
        if cursor.fetchone()[0] == 0:
            # If department does not exist, add it
            dept_name = input("Department not found. Enter the department name to add it: ").strip()
            if not dept_name:
                print("Error: Department name cannot be empty.")
                cursor.execute("ROLLBACK;")
                return
            cursor.execute("INSERT INTO DEPARTMENT (dept_code, name) VALUES (%s, %s);", (dept_code, dept_name))
            print(f"Department '{dept_name}' added with code '{dept_code}'.")
        
        # Prompt user for degree details
        degree_name = input("Enter the degree name (e.g., Computer Science): ").strip()
        degree_level = input("Enter the degree level (e.g., BA, BS, MS, PhD, Cert): ").strip()
        if not degree_name or not degree_level:
            print("Error: Degree name and level cannot be empty.")
            cursor.execute("ROLLBACK;")
            return
        
        # Check for duplicate degrees
        cursor.execute(
            "SELECT COUNT(*) FROM DEGREE WHERE dept_code = %s AND name = %s AND level = %s;",
            (dept_code, degree_name, degree_level)
        )
        if cursor.fetchone()[0] > 0:
            print("Error: A degree with the same name and level already exists in this department.")
            cursor.execute("ROLLBACK;")
            return
        
        # Insert the new degree
        cursor.execute(
            "INSERT INTO DEGREE (dept_code, name, level) VALUES (%s, %s, %s);",
            (dept_code, degree_name, degree_level)
        )
        print(f"Degree '{degree_name}' at level '{degree_level}' added to department '{dept_code}'.")
        
        # Commit the transaction
        cursor.execute("COMMIT;")
    
    except Exception as e:
        print(f"An error occurred: {e}")
        cursor.execute("ROLLBACK;")
        sys.exit(1)


## Enter Course

In [9]:
def add_course(cursor):
    """
    Allows the user to add a new course and assign it to multiple degrees.
    """
    try:
        # Start a transaction
        cursor.execute("BEGIN;")
        
        print("Enter the course details:")
        
        # Validate course number format
        while True:
            course_number = input("Course Number (e.g., CS1010): ").strip().upper()
            if re.fullmatch(r'[A-Z]{2,4}\d{4}', course_number):
                break
            else:
                print("Error: Course number must consist of 2-4 uppercase letters followed by 4 digits (e.g., CS1010).")
        
        # Validate department code
        while True:
            dept_code = input("Department Code (2-4 characters, e.g., CS): ").strip().upper()
            if re.fullmatch(r'[A-Z]{2,4}', dept_code):
                break
            else:
                print("Error: Department code must be 2-4 uppercase characters (e.g., CS).")
        
        course_name = input("Course Name: ").strip()
        if not course_name:
            print("Error: Course name cannot be empty.")
            cursor.execute("ROLLBACK;")
            return
        
        # Check if department exists
        cursor.execute("SELECT COUNT(*) FROM DEPARTMENT WHERE dept_code = %s;", (dept_code,))
        if cursor.fetchone()[0] == 0:
            print(f"Error: Department '{dept_code}' does not exist. Please add it first.")
            cursor.execute("ROLLBACK;")
            return
        
        # Check for unique course number
        cursor.execute("SELECT COUNT(*) FROM COURSE WHERE course_number = %s;", (course_number,))
        if cursor.fetchone()[0] > 0:
            print(f"Error: Course '{course_number}' already exists.")
            cursor.execute("ROLLBACK;")
            return
        
        # Insert course
        cursor.execute(
            "INSERT INTO COURSE (course_number, dept_code, name) VALUES (%s, %s, %s);",
            (course_number, dept_code, course_name)
        )
        print(f"Course '{course_number}' added successfully.")
        
        # Ask if the course is core for any degree
        while True:
            add_to_degree = input("Do you want to assign this course to a degree? (yes/no): ").strip().lower()
            if add_to_degree == 'no':
                break
            elif add_to_degree == 'yes':
                # Get degree details
                degree_name = input("Enter the degree name: ").strip()
                degree_level = input("Enter the degree level (e.g., Bachelor, Master): ").strip()
                
                # Check if the degree exists
                cursor.execute("""
                    SELECT COUNT(*) 
                    FROM DEGREE 
                    WHERE dept_code = %s AND name = %s AND level = %s;
                """, (dept_code, degree_name, degree_level))
                if cursor.fetchone()[0] == 0:
                    print(f"Error: Degree '{degree_name}' ({degree_level}) does not exist.")
                    continue
                
                # Check if the course is already assigned to the degree
                cursor.execute("""
                    SELECT COUNT(*) 
                    FROM DEGREE_COURSE 
                    WHERE dept_code = %s AND degree_name = %s AND degree_level = %s AND course_number = %s;
                """, (dept_code, degree_name, degree_level, course_number))
                if cursor.fetchone()[0] > 0:
                    print("This course is already assigned to this degree.")
                    continue
                
                # Ask if the course is a core course
                is_core = input("Is this course a core course for the degree? (yes/no): ").strip().lower() == 'yes'
                
                # Add the course to the degree
                cursor.execute("""
                    INSERT INTO DEGREE_COURSE (dept_code, degree_name, degree_level, course_number, is_core)
                    VALUES (%s, %s, %s, %s, %s);
                """, (dept_code, degree_name, degree_level, course_number, is_core))
                print(f"Course '{course_number}' added to degree '{degree_name}' ({degree_level}).")
            else:
                print("Invalid input. Please enter 'yes' or 'no'.")
        
        # Commit the transaction
        cursor.execute("COMMIT;")
    
    except Exception as e:
        print(f"An error occurred: {e}")
        cursor.execute("ROLLBACK;")
        sys.exit(1)


## Enter Section

In [11]:
def add_section(cursor):
    """
    Prompts the user to enter information about sections and ensures all components are validated
    and related tables are updated as necessary.
    """
    try:
        # Begin a transaction
        cursor.execute("BEGIN;")

        print("Enter details for the new section:")

        # Input course number and validate it exists
        while True:
            course_number = input("Course Number (e.g., CS1010): ").strip().upper()
            if re.fullmatch(r'[A-Z]{2,4}\d{4}', course_number):
                cursor.execute("SELECT COUNT(*) FROM COURSE WHERE course_number = %s;", (course_number,))
                if cursor.fetchone()[0] > 0:
                    break
                else:
                    print(f"Error: Course '{course_number}' does not exist. Please add it first.")
            else:
                print("Error: Invalid course number format. Use 2-4 uppercase letters followed by 4 digits.")

        # Input semester details and validate or insert into SEMESTER
        while True:
            year = input("Year (e.g., 2024): ").strip()
            if year.isdigit() and 1900 <= int(year) <= 2100:
                year = int(year)
                break
            else:
                print("Error: Year must be a valid 4-digit number between 1900 and 2100.")

        while True:
            term = input("Term (Spring, Summer, Fall): ").strip().capitalize()
            if term in {'Spring', 'Summer', 'Fall'}:
                # Check if the semester exists, insert if not
                cursor.execute("SELECT COUNT(*) FROM SEMESTER WHERE year = %s AND term = %s;", (year, term))
                if cursor.fetchone()[0] == 0:
                    cursor.execute("INSERT INTO SEMESTER (year, term) VALUES (%s, %s);", (year, term))
                    print(f"Semester '{year} {term}' added.")
                break
            else:
                print("Error: Term must be one of 'Spring', 'Summer', or 'Fall'.")

        # Input section number and ensure it is unique
        while True:
            section_number = input("Section Number (3 digits, e.g., 001): ").strip()
            if re.fullmatch(r'\d{3}', section_number):
                cursor.execute(
                    "SELECT COUNT(*) FROM SECTION WHERE course_number = %s AND year = %s AND term = %s AND section_number = %s;",
                    (course_number, year, term, section_number)
                )
                if cursor.fetchone()[0] == 0:
                    break
                else:
                    print(f"Error: Section '{section_number}' for course '{course_number}' in '{year} {term}' already exists.")
            else:
                print("Error: Section number must be exactly 3 digits.")

        # Input enrollment and validate
        while True:
            enrollment = input("Number of students enrolled: ").strip()
            if enrollment.isdigit() and int(enrollment) >= 0:
                enrollment = int(enrollment)
                break
            else:
                print("Error: Enrollment must be a non-negative integer.")

        # Input instructor ID and validate
        while True:
            instructor_id = input("Instructor ID (8 characters): ").strip().upper()
            if re.fullmatch(r'[A-Z0-9]{8}', instructor_id):
                cursor.execute("SELECT COUNT(*) FROM INSTRUCTOR WHERE instructor_id = %s;", (instructor_id,))
                if cursor.fetchone()[0] > 0:
                    break
                else:
                    print(f"Error: Instructor '{instructor_id}' does not exist. Please add them first.")
            else:
                print("Error: Instructor ID must be exactly 8 characters (letters or digits).")

        # Insert into SECTION table
        cursor.execute(
            """INSERT INTO SECTION (course_number, year, term, section_number, enrollment, instructor_id)
               VALUES (%s, %s, %s, %s, %s, %s);""",
            (course_number, year, term, section_number, enrollment, instructor_id)
        )
        print(f"Section '{section_number}' for course '{course_number}' in '{year} {term}' added successfully.")

        # Commit the transaction
        cursor.execute("COMMIT;")
    except Exception as e:
        print(f"An error occurred: {e}")
        cursor.execute("ROLLBACK;")
        sys.exit(1)


## Add Instructor

In [13]:
def add_instructor(cursor):
    """
    Prompts the user to input information about an instructor and inserts the data into the INSTRUCTOR table.
    Ensures that the department exists before adding the instructor.
    """
    while True:
        try:
            # Prompt user for instructor details
            instructor_id = input("Enter the instructor's 8-digit ID: ").strip()
            if len(instructor_id) != 8 or not instructor_id.isdigit():
                raise ValueError("Instructor ID must be an 8-digit number.")
            
            name = input("Enter the instructor's name: ").strip()
            if not name:
                raise ValueError("Instructor name cannot be empty.")
            
            dept_code = input("Enter the department code (2-4 characters): ").strip().upper()
            if not re.fullmatch(r'[A-Z]{2,4}', dept_code):
                print("Error: Department code must be 2-4 uppercase characters.")
                cursor.execute("ROLLBACK;")
                return
            
            # Check if the department exists
            cursor.execute("SELECT * FROM DEPARTMENT WHERE dept_code = %s", (dept_code,))
            if cursor.fetchone() is None:
                print(f"Department '{dept_code}' does not exist. Please add the department first.")
                continue
            
            # Check if instructor ID already exists
            cursor.execute("SELECT * FROM INSTRUCTOR WHERE instructor_id = %s", (instructor_id,))
            if cursor.fetchone():
                print(f"Instructor ID '{instructor_id}' already exists. Please enter a unique ID.")
                continue
            
            # Insert instructor into the INSTRUCTOR table
            cursor.execute(
                "INSERT INTO INSTRUCTOR (instructor_id, name, dept_code) VALUES (%s, %s, %s)",
                (instructor_id, name, dept_code)
            )
            print(f"Instructor '{name}' added successfully under department '{dept_code}'.")
            break
        except ValueError as e:
            print(f"Error: {e}")

## Enter Goal

In [15]:
def add_goal(cursor):
    """
    Prompts the user to input information about a goal and inserts the data into the GOAL table.
    Ensures that the degree associated with the goal exists.
    """
    while True:
        try:
            # Prompt user for goal details
            dept_code = input("Enter the department code (2-4 characters): ").strip().upper()
            if not re.fullmatch(r'[A-Z]{2,4}', dept_code):
                print("Error: Department code must be 2-4 uppercase characters.")
                cursor.execute("ROLLBACK;")
                return

            degree_name = input("Enter the degree name: ").strip()
            if not degree_name:
                raise ValueError("Degree name cannot be empty.")

            degree_level = input("Enter the degree level (e.g., BA, BS, MS, PhD, Cert): ").strip()

            goal_code = input("Enter the goal code (4 characters): ").strip()
            if len(goal_code) != 4:
                raise ValueError("Goal code must be exactly 4 characters.")

            description = input("Enter the goal description: ").strip()
            if not description:
                raise ValueError("Goal description cannot be empty.")

            # Check if the degree exists
            cursor.execute(
                "SELECT * FROM DEGREE WHERE dept_code = %s AND name = %s AND level = %s",
                (dept_code, degree_name, degree_level)
            )
            if cursor.fetchone() is None:
                print(f"Degree '{degree_name}' at level '{degree_level}' in department '{dept_code}' does not exist.")
                print("Please add the degree first.")
                continue

            # Check if the goal code already exists for the degree
            cursor.execute(
                "SELECT * FROM GOAL WHERE dept_code = %s AND degree_name = %s AND degree_level = %s AND code = %s",
                (dept_code, degree_name, degree_level, goal_code)
            )
            if cursor.fetchone():
                print(f"Goal code '{goal_code}' already exists for the degree '{degree_name}' in department '{dept_code}'.")
                continue

            # Insert the goal into the GOAL table
            cursor.execute(
                "INSERT INTO GOAL (dept_code, degree_name, degree_level, code, description) VALUES (%s, %s, %s, %s, %s)",
                (dept_code, degree_name, degree_level, goal_code, description)
            )
            print(f"Goal '{goal_code}' added successfully to degree '{degree_name}' in department '{dept_code}'.")
            break
        except ValueError as e:
            print(f"Error: {e}")

## Associate Course and  Goal

In [17]:
def associate_course_with_goal(cursor):
    """
    Associates a course with a goal by linking it through the DEGREE_COURSE table.
    Ensures all necessary constraints are validated.
    """
    while True:
        try:
            # Prompt user for required inputs
            dept_code = input("Enter the department code (2-4 characters): ").strip().upper()
            if not re.fullmatch(r'[A-Z]{2,4}', dept_code):
                print("Error: Department code must be 2-4 uppercase characters.")
                cursor.execute("ROLLBACK;")
                return

            degree_name = input("Enter the degree name: ").strip()
            if not degree_name:
                raise ValueError("Degree name cannot be empty.")

            degree_level = input("Enter the degree level (e.g., BA, BS, MS, PhD, Cert): ").strip()

            course_number = input("Enter the course number (e.g., CS101): ").strip()
            if not course_number:
                raise ValueError("Course number cannot be empty.")

            goal_code = input("Enter the goal code (4 characters): ").strip()
            if len(goal_code) != 4:
                raise ValueError("Goal code must be exactly 4 characters.")

            # Validate that the degree exists
            cursor.execute(
                "SELECT * FROM DEGREE WHERE dept_code = %s AND name = %s AND level = %s",
                (dept_code, degree_name, degree_level)
            )
            if cursor.fetchone() is None:
                print(f"Degree '{degree_name}' at level '{degree_level}' in department '{dept_code}' does not exist.")
                print("Please add the degree first.")
                continue

            # Validate that the course exists
            cursor.execute(
                "SELECT * FROM COURSE WHERE course_number = %s AND dept_code = %s",
                (course_number, dept_code)
            )
            if cursor.fetchone() is None:
                print(f"Course '{course_number}' in department '{dept_code}' does not exist.")
                print("Please add the course first.")
                continue

            # Validate that the goal exists
            cursor.execute(
                "SELECT * FROM GOAL WHERE dept_code = %s AND degree_name = %s AND degree_level = %s AND code = %s",
                (dept_code, degree_name, degree_level, goal_code)
            )
            if cursor.fetchone() is None:
                print(f"Goal '{goal_code}' for degree '{degree_name}' in department '{dept_code}' does not exist.")
                print("Please add the goal first.")
                continue

            # Check if the course is already associated with the degree
            cursor.execute(
                "SELECT * FROM DEGREE_COURSE WHERE dept_code = %s AND degree_name = %s AND degree_level = %s AND course_number = %s",
                (dept_code, degree_name, degree_level, course_number)
            )
            if cursor.fetchone() is None:
                # Associate the course with the degree in DEGREE_COURSE
                cursor.execute(
                    "INSERT INTO DEGREE_COURSE (dept_code, degree_name, degree_level, course_number, is_core) "
                    "VALUES (%s, %s, %s, %s, %s)",
                    (dept_code, degree_name, degree_level, course_number, False)
                )
                print(f"Course '{course_number}' has been associated with degree '{degree_name}' in department '{dept_code}'.")

            # The association is valid; no further action required for the GOAL table
            print(f"Course '{course_number}' successfully associated with goal '{goal_code}' for degree '{degree_name}'.")
            break

        except ValueError as e:
            print(f"Input error: {e}")

## Enter Course/Section Given Semester

In [19]:
def enter_course_section_for_semester(cursor):
    """
    Adds a course section for a given semester.
    Ensures the course exists, the semester exists, and the instructor is valid.
    """
    while True:
        try:
            # Prompt the user for necessary information
            dept_code = input("Enter the department code (2-4 characters): ").strip().upper()
            if not re.fullmatch(r'[A-Z]{2,4}', dept_code):
                print("Error: Department code must be 2-4 uppercase characters.")
                cursor.execute("ROLLBACK;")
                return

            course_number = input("Enter the course number (e.g., CS101): ").strip()
            if not course_number:
                raise ValueError("Course number cannot be empty.")

            # Validate that the course exists in the COURSE table
            cursor.execute(
                "SELECT * FROM COURSE WHERE course_number = %s AND dept_code = %s",
                (course_number, dept_code)
            )
            if cursor.fetchone() is None:
                print(f"Course '{course_number}' in department '{dept_code}' does not exist.")
                print("Please add the course first.")
                continue

            while True:
                year = input("Year (e.g., 2024): ").strip()
                if year.isdigit() and 1900 <= int(year) <= 2100:
                    year = int(year)
                    break
                else:
                    print("Error: Year must be a valid 4-digit number between 1900 and 2100.")

            while True:
                term = input("Term (Spring, Summer, Fall): ").strip().capitalize()
                if term in {'Spring', 'Summer', 'Fall'}:
                    # Check if the semester exists, insert if not
                    cursor.execute("SELECT COUNT(*) FROM SEMESTER WHERE year = %s AND term = %s;", (year, term))
                    if cursor.fetchone()[0] == 0:
                        cursor.execute("INSERT INTO SEMESTER (year, term) VALUES (%s, %s);", (year, term))
                        print(f"Semester '{year} {term}' added.")
                    break
                else:
                    print("Error: Term must be one of 'Spring', 'Summer', or 'Fall'.")

            # Prompt for section information
            section_number = input("Enter the section number (e.g., 001): ").strip()
            if len(section_number) != 3:
                raise ValueError("Section number must be 3 characters.")

            # Prompt for instructor information
            instructor_id = input("Enter the instructor ID: ").strip()
            if len(instructor_id) != 8:
                raise ValueError("Instructor ID must be exactly 8 characters.")

            # Validate that the instructor exists in the INSTRUCTOR table
            cursor.execute(
                "SELECT * FROM INSTRUCTOR WHERE instructor_id = %s AND dept_code = %s",
                (instructor_id, dept_code)
            )
            if cursor.fetchone() is None:
                print(f"Instructor with ID '{instructor_id}' does not exist in department '{dept_code}'.")
                print("Please add the instructor first.")
                continue

            # Check if the section already exists for the given course, year, term, and section number
            cursor.execute(
                "SELECT * FROM SECTION WHERE course_number = %s AND year = %s AND term = %s AND section_number = %s",
                (course_number, year, term, section_number)
            )
            if cursor.fetchone() is not None:
                print(f"Section {section_number} for course {course_number} in {term} {year} already exists.")
                continue

            # Insert the new section into the SECTION table
            cursor.execute(
                "INSERT INTO SECTION (course_number, year, term, section_number, instructor_id, enrollment) "
                "VALUES (%s, %s, %s, %s, %s, 0)",
                (course_number, year, term, section_number, instructor_id)
            )

            print(f"Section {section_number} for course '{course_number}' in {term} {year} has been added.")
            break

        except ValueError as e:
            print(f"Input error: {e}")

# Querying

## Given degree, List Courses

In [22]:
def list_courses_for_degree(cursor):
    """
    Lists all courses associated with a given degree and indicates which are core.
    """
    try:
        # Prompt user for degree details
        dept_code = input("Enter the department code (e.g., CS): ").strip().upper()
        degree_name = input("Enter the degree name (e.g., Computer Science): ").strip()
        degree_level = input("Enter the degree level (e.g., BA, MS): ").strip()

        if not dept_code or not degree_name or not degree_level:
            print("Error: Department code, degree name, and degree level cannot be empty.")
            return
        
        # Validate that the degree exists
        cursor.execute(
            "SELECT * FROM DEGREE WHERE dept_code = %s AND name = %s AND level = %s",
            (dept_code, degree_name, degree_level)
        )
        if cursor.fetchone() is None:
            print(f"Error: Degree '{degree_name}' at level '{degree_level}' in department '{dept_code}' does not exist.")
            return
        
        # Query to get all courses associated with the degree
        cursor.execute("""
            SELECT c.course_number, c.name, dc.is_core
            FROM COURSE c
            JOIN DEGREE_COURSE dc ON c.course_number = dc.course_number
            WHERE dc.dept_code = %s AND dc.degree_name = %s AND dc.degree_level = %s
            ORDER BY c.name;
        """, (dept_code, degree_name, degree_level))

        courses = cursor.fetchall()
        
        if not courses:
            print(f"No courses found for degree '{degree_name}' at level '{degree_level}' in department '{dept_code}'.")
            return
        
        # Print the course list with core status
        print(f"\nCourses for Degree '{degree_name}' (Level: {degree_level}) in Department '{dept_code}':")
        for course in courses:
            core_status = "Core" if course[2] else "Elective"
            print(f"Course Number: {course[0]}, Course Name: {course[1]}, Status: {core_status}")

    except Exception as e:
        print(f"An error occurred: {e}")

## Given Degree, List Sections

In [24]:
def list_sections_by_degree(cursor, start_year, end_year, start_term, end_term):
    try:
        # Prompt user for degree details
        dept_code = input("Enter the department code (e.g., CS): ").strip().upper()
        degree_name = input("Enter the degree name (e.g., Computer Science): ").strip()
        degree_level = input("Enter the degree level (e.g., BA, MS): ").strip()

        # Validate input for terms
        valid_terms = ['Spring', 'Summer', 'Fall']
        if start_term not in valid_terms or end_term not in valid_terms:
            print("Error: Terms must be one of 'Spring', 'Summer', or 'Fall'.")
            return

        # Validate degree exists
        cursor.execute(""" 
        SELECT COUNT(*) 
        FROM DEGREE 
        WHERE dept_code = %s AND name = %s AND level = %s; 
        """, (dept_code, degree_name, degree_level))
        
        if cursor.fetchone()[0] == 0:
            print(f"Error: Degree '{degree_name}' at level '{degree_level}' in department '{dept_code}' does not exist.")
            return
        
        # Query to list all sections for the specified degree and time range, ordered by year and term
        query = """
        SELECT sec.course_number, sec.year, sec.term, sec.section_number, sec.enrollment, sec.instructor_id,
               c.name AS course_name, i.name AS instructor_name
        FROM SECTION sec
        JOIN COURSE c ON sec.course_number = c.course_number
        JOIN INSTRUCTOR i ON sec.instructor_id = i.instructor_id
        JOIN DEGREE_COURSE dc ON c.course_number = dc.course_number
        WHERE dc.degree_name = %s AND dc.degree_level = %s
        AND (sec.year > %s OR (sec.year = %s AND sec.term >= %s))
        AND (sec.year < %s OR (sec.year = %s AND sec.term <= %s))
        ORDER BY sec.year ASC, FIELD(sec.term, 'Spring', 'Summer', 'Fall') ASC, sec.section_number ASC;
        """
        
        cursor.execute(query, (degree_name, degree_level, start_year, start_year, start_term, end_year, end_year, end_term))
        
        # Fetch and print results
        sections = cursor.fetchall()
        if not sections:
            print("No sections found within the specified time range.")
        else:
            print(f"Sections offered for '{degree_name}' degree ({degree_level}) from {start_term} {start_year} to {end_term} {end_year}:")
            for section in sections:
                print(f"Course: {section[6]} | Section: {section[3]} | Instructor: {section[7]} | "
                      f"Year: {section[1]} | Term: {section[2]} | Enrollment: {section[4]}")
    except Exception as e:
        print(f"An error occurred: {e}")


## Given Degree, List all Goals

In [26]:
def list_goals(cursor):
    """
    Lists all goals for a specified degree program.
    Ensures the degree exists and handles invalid or missing inputs gracefully.
    """
    try:
        # Prompt user for degree details
        dept_code = input("Enter the department code (2-4 characters): ").strip().upper()
        if not re.fullmatch(r'[A-Z]{2,4}', dept_code):
            print("Error: Department code must be 2-4 uppercase characters.")
            return

        degree_name = input("Enter the degree name: ").strip()
        if not degree_name:
            print("Error: Degree name cannot be empty.")
            return

        degree_level = input("Enter the degree level (e.g., BA, BS, MS, PhD, Cert): ").strip()
        if not degree_level:
            print("Error: Degree level cannot be empty.")
            return

        # Check if the degree exists
        cursor.execute(
            "SELECT COUNT(*) FROM DEGREE WHERE dept_code = %s AND name = %s AND level = %s;",
            (dept_code, degree_name, degree_level)
        )
        if cursor.fetchone()[0] == 0:
            print(f"Error: Degree '{degree_name}' at level '{degree_level}' in department '{dept_code}' does not exist.")
            return

        # Retrieve and display goals for the degree
        cursor.execute(
            """
            SELECT code, description
            FROM GOAL
            WHERE dept_code = %s AND degree_name = %s AND degree_level = %s
            ORDER BY code;
            """,
            (dept_code, degree_name, degree_level)
        )
        goals = cursor.fetchall()

        if not goals:
            print(f"No goals found for degree '{degree_name}' at level '{degree_level}' in department '{dept_code}'.")
        else:
            print(f"Goals for degree '{degree_name}' at level '{degree_level}' in department '{dept_code}':")
            for code, description in goals:
                print(f"- {code}: {description}")
    except Exception as e:
        print(f"An error occurred while listing goals: {e}")


## Given Degree, List Courses Associated with Goal

In [28]:
def list_courses_by_goals(cursor):
    """
    Lists courses associated with each goal for a given degree.
    Allows the user to specify the degree and optionally filter by specific goals.
    """
    try:
        # Prompt for degree details
        dept_code = input("Enter the department code (2-4 characters): ").strip().upper()
        degree_name = input("Enter the degree name: ").strip()
        degree_level = input("Enter the degree level (e.g., BA, BS, MS, PhD, Cert): ").strip()

        # Validate degree existence
        cursor.execute("""
            SELECT * FROM DEGREE 
            WHERE dept_code = %s AND name = %s AND level = %s;
        """, (dept_code, degree_name, degree_level))
        degree = cursor.fetchone()

        if not degree:
            print(f"Error: Degree '{degree_name}' at level '{degree_level}' in department '{dept_code}' does not exist.")
            return

        # Ask the user if they want to filter by specific goals
        filter_goals = input("Do you want to filter by specific goals? (yes/no): ").strip().lower()
        if filter_goals == 'yes':
            goal_codes = input("Enter the goal codes (comma-separated, e.g., GO01,GO02): ").strip().split(',')
            goal_codes = [code.strip().upper() for code in goal_codes]
            query = """
                SELECT G.code AS goal_code, G.description AS goal_description, C.course_number, C.name AS course_name
                FROM GOAL G
                JOIN DEGREE_COURSE DC ON G.dept_code = DC.dept_code 
                    AND G.degree_name = DC.degree_name 
                    AND G.degree_level = DC.degree_level
                JOIN COURSE C ON DC.course_number = C.course_number
                WHERE G.dept_code = %s AND G.degree_name = %s AND G.degree_level = %s
                  AND G.code IN %s;
            """
            cursor.execute(query, (dept_code, degree_name, degree_level, tuple(goal_codes)))
        else:
            query = """
                SELECT G.code AS goal_code, G.description AS goal_description, C.course_number, C.name AS course_name
                FROM GOAL G
                JOIN DEGREE_COURSE DC ON G.dept_code = DC.dept_code 
                    AND G.degree_name = DC.degree_name 
                    AND G.degree_level = DC.degree_level
                JOIN COURSE C ON DC.course_number = C.course_number
                WHERE G.dept_code = %s AND G.degree_name = %s AND G.degree_level = %s;
            """
            cursor.execute(query, (dept_code, degree_name, degree_level))

        # Fetch and display results
        results = cursor.fetchall()
        if results:
            print(f"Courses associated with goals for degree '{degree_name}' ({degree_level}) in department '{dept_code}':")
            for row in results:
                print(f"- Goal Code: {row['goal_code']}, Description: {row['goal_description']}, "
                      f"Course: {row['course_number']} ({row['course_name']})")
        else:
            print("No courses found for the specified degree and goals.")

    except Exception as e:
        print(f"An error occurred: {e}")


## Given a Course, List all Sections within Range

In [30]:
def list_course_sections(cursor, course_number, start_year, start_term, end_year, end_term):
    """
    Lists all sections of a course within a specific range of semesters.

    Parameters:
        cursor (MySQLCursor): The database cursor to execute SQL queries.
        course_number (str): The course number (e.g., 'CS1010').
        start_year (int): The starting year of the range.
        start_term (str): The starting term of the range ('Spring', 'Summer', 'Fall').
        end_year (int): The ending year of the range.
        end_term (str): The ending term of the range.

    Returns:
        None: Prints the sections found or an appropriate message if no sections exist.
    """
    try:
        # Validate terms
        valid_terms = {'Spring': 1, 'Summer': 2, 'Fall': 3}
        if start_term not in valid_terms or end_term not in valid_terms:
            print("Error: Terms must be one of 'Spring', 'Summer', or 'Fall'.")
            return

        # Ensure the date range is valid
        if (start_year > end_year) or (start_year == end_year and valid_terms[start_term] > valid_terms[end_term]):
            print("Error: The start semester must come before the end semester.")
            return

        # Query to fetch sections within the range
        query = """
            SELECT year, term, section_number, enrollment, instructor_id
            FROM SECTION
            WHERE course_number = %s
              AND (year > %s OR (year = %s AND FIELD(term, 'Spring', 'Summer', 'Fall') >= FIELD(%s, 'Spring', 'Summer', 'Fall')))
              AND (year < %s OR (year = %s AND FIELD(term, 'Spring', 'Summer', 'Fall') <= FIELD(%s, 'Spring', 'Summer', 'Fall')))
            ORDER BY year, FIELD(term, 'Spring', 'Summer', 'Fall'), section_number;
        """
        cursor.execute(query, (course_number, start_year, start_year, start_term, end_year, end_year, end_term))
        sections = cursor.fetchall()

        # Print results
        if sections:
            print(f"Sections for course '{course_number}' from {start_year} {start_term} to {end_year} {end_term}:")
            for year, term, section_number, enrollment, instructor_id in sections:
                print(f"  Year: {year}, Term: {term}, Section: {section_number}, Enrollment: {enrollment}, Instructor ID: {instructor_id}")
        else:
            print(f"No sections found for course '{course_number}' in the specified range.")

    except Exception as e:
        print(f"An error occurred while fetching sections: {e}")


## Given Instructor, List Sections

In [32]:

def list_sections_by_instructor(cursor, instructor_id, start_year, start_term, end_year, end_term):
    """
    Lists all sections taught by a specific instructor within a specified range of semesters.
    
    Args:
        cursor: Database cursor for executing queries.
        instructor_id: The ID of the instructor (8 characters).
        start_year: The start year of the range (integer).
        start_term: The start term of the range ('Spring', 'Summer', or 'Fall').
        end_year: The end year of the range (integer).
        end_term: The end term of the range ('Spring', 'Summer', or 'Fall').
        
    Returns:
        A list of tuples containing section details (course_number, year, term, section_number).
    """
    try:
        # Validate terms
        valid_terms = {'Spring': 1, 'Summer': 2, 'Fall': 3}
        if start_term not in valid_terms or end_term not in valid_terms:
            raise ValueError("Terms must be one of 'Spring', 'Summer', or 'Fall'.")

        # Convert terms to sortable values
        start_semester = (start_year, valid_terms[start_term])
        end_semester = (end_year, valid_terms[end_term])

        if start_semester > end_semester:
            raise ValueError("Start semester must not come after end semester.")

        # Query sections taught by the instructor within the range
        cursor.execute("""
            SELECT course_number, year, term, section_number
            FROM SECTION
            WHERE instructor_id = %s
              AND ((year > %s) OR (year = %s AND term >= %s))
              AND ((year < %s) OR (year = %s AND term <= %s))
            ORDER BY year, term, section_number;
        """, (
            instructor_id, 
            start_year, start_year, valid_terms[start_term], 
            end_year, end_year, valid_terms[end_term]
        ))

        # Fetch results
        sections = cursor.fetchall()

        # Display results or indicate no records found
        if sections:
            print(f"Sections taught by instructor '{instructor_id}':")
            for section in sections:
                print(f" - Course: {section[0]}, Year: {section[1]}, Term: {section[2]}, Section: {section[3]}")
        else:
            print(f"No sections found for instructor '{instructor_id}' in the specified range.")

        return sections

    except Exception as e:
        print(f"An error occurred: {e}")


## main()

In [34]:
import mysql.connector
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

def main():
    cnx = None
    cursor = None

    try:
        # Database connection setup
        cnx = mysql.connector.connect(
            user=os.getenv('MYSQL_USER'),
            password=os.getenv('MYSQL_PASSWORD'),
            host=os.getenv('MYSQL_HOST'),
            database=os.getenv('MYSQL_DATABASE')
        )
        cursor = cnx.cursor()
        print("Database connection established.")

        check_create_tables(cursor)  # Ensure tables exist before any interaction

        # Menu loop
        while True:
            print("\n--- Menu ---")
            print("1. Enter Degree")
            print("2. Enter Course")
            print("3. Enter Section")
            print("4. Enter Instructor")
            print("5. Enter Goal")
            print("6. Associate Course with Goal")
            print("7. Enter Course Section for Semester")
            print("8. List Courses for Degree")
            print("9. List Sections by Degree (within date range)")
            print("10. List Goals")
            print("11. List Courses by Goals")
            print("12. List Course Sections (within date range)")
            print("13. List Sections by Instructor (within date range)")

            print("\n14. Clear all Tables")
            print("15. Quit")

            command = input("\nChoose an option (1-14): ").strip()

            if command == '15':
                print("Exiting the program...")
                break

            try:
                if command == '1':
                    add_degree(cursor)
                elif command == '2':
                    add_course(cursor)
                elif command == '3':
                    add_section(cursor)
                elif command == '4':
                    add_instructor(cursor)
                elif command == '5':
                    add_goal(cursor)
                elif command == '6':
                    associate_course_with_goal(cursor)
                elif command == '7':
                    enter_course_section_for_semester(cursor)
                elif command == '8':
                    list_courses_for_degree(cursor)
                elif command == '9':
                    start_year = int(input("Enter start year: "))
                    end_year = int(input("Enter end year: "))
                    start_term = input("Enter start term (Spring, Summer, Fall): ").capitalize()
                    end_term = input("Enter end term (Spring, Summer, Fall): ").capitalize()
                    list_sections_by_degree(cursor, start_year, end_year, start_term, end_term)
                elif command == '10':
                    list_goals(cursor)
                elif command == '11':
                    list_courses_by_goals(cursor)
                elif command == '12':
                    course_number = input("Enter course number: ").strip().upper()
                    start_year = int(input("Enter start year: "))
                    start_term = input("Enter start term (Spring, Summer, Fall): ").capitalize()
                    end_year = int(input("Enter end year: "))
                    end_term = input("Enter end term (Spring, Summer, Fall): ").capitalize()
                    list_course_sections(cursor, course_number, start_year, start_term, end_year, end_term)
                elif command == '13':
                    instructor_id = input("Enter instructor ID: ").strip()
                    start_year = int(input("Enter start year: "))
                    start_term = input("Enter start term (Spring, Summer, Fall): ").capitalize()
                    end_year = int(input("Enter end year: "))
                    end_term = input("Enter end term (Spring, Summer, Fall): ").capitalize()
                    list_sections_by_instructor(cursor, instructor_id, start_year, start_term, end_year, end_term)
                elif command == '14':
                    clear_data(cursor)
                else:
                    print("Invalid option. Please select a number between 1 and 15.")
            except Exception as e:
                print(f"An error occurred while processing your request: {e}")

    except mysql.connector.Error as err:  # Updated error handling reference
        print(f"Database error: {err}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        # Close the cursor and connection
        if cursor is not None:
            try:
                cursor.close()
                print("Cursor closed.")
            except mysql.connector.Error as err:  # Updated error handling reference
                print(f"Error closing cursor: {err}")
            except Exception as e:
                print(f"Unexpected error while closing cursor: {e}")

        if cnx is not None:
            try:
                cnx.close()
                print("Database connection closed.")
            except mysql.connector.Error as err:  # Updated error handling reference
                print(f"Error closing connection: {err}")
            except Exception as e:
                print(f"Unexpected error while closing connection: {e}")

if __name__ == "__main__":
    main()

Database connection established.

--- Menu ---
1. Enter Degree
2. Enter Course
3. Enter Section
4. Enter Instructor
5. Enter Goal
6. Associate Course with Goal
7. Enter Course Section for Semester
8. List Courses for Degree
9. List Sections by Degree (within date range)
10. List Goals
11. List Courses by Goals
12. List Course Sections (within date range)
13. List Sections by Instructor (within date range)

14. Clear all Tables
15. Quit



Choose an option (1-14):  9
Enter start year:  2022
Enter end year:  2024
Enter start term (Spring, Summer, Fall):  fall
Enter end term (Spring, Summer, Fall):  fall
Enter the department code (e.g., CS):  CS
Enter the degree name (e.g., Computer Science):  Computer Science
Enter the degree level (e.g., BA, MS):  BS


No sections found within the specified time range.

--- Menu ---
1. Enter Degree
2. Enter Course
3. Enter Section
4. Enter Instructor
5. Enter Goal
6. Associate Course with Goal
7. Enter Course Section for Semester
8. List Courses for Degree
9. List Sections by Degree (within date range)
10. List Goals
11. List Courses by Goals
12. List Course Sections (within date range)
13. List Sections by Instructor (within date range)

14. Clear all Tables
15. Quit



Choose an option (1-14):  2


Enter the course details:


...

Course 'CS5330' added to degree 'Computer Science' (BS).


Do you want to assign this course to a degree? (yes/no):  no



--- Menu ---
1. Enter Degree
2. Enter Course
3. Enter Section
4. Enter Instructor
5. Enter Goal
6. Associate Course with Goal
7. Enter Course Section for Semester
8. List Courses for Degree
9. List Sections by Degree (within date range)
10. List Goals
11. List Courses by Goals
12. List Course Sections (within date range)
13. List Sections by Instructor (within date range)

14. Clear all Tables
15. Quit



Choose an option (1-14):  15


Exiting the program...
Cursor closed.
Database connection closed.
